In [3]:
# main.py
from lark import Lark
from parser import AstTransformer, dump_symTab, dump_z3_structs, reset_parser_globals
from LABSCore import *
from utils import *

reset_parser_globals()

with open('examples/sendMessage.labs', 'r') as structTest:
    # The source code to be parsed
    source_code = structTest.read()

with open('grammar.lark', 'r') as f:
    grammar = f.read()

transformer = AstTransformer()
parser = Lark(grammar, start='start', parser='lalr', transformer=transformer)

print("--- Parsing Structured Program ---")
# The parser now returns a tuple
program_ast = parser.parse(source_code)
print("Parsing successful!")

print("\n--- Precondition ---")
print((fmla_enc(program_ast.precondition)))

print("\nGenerated Postconditions\n")

Q = post(program_ast.program, fmla_enc(program_ast.precondition))

print((Q), "\n\n")

print("\nExpected Postcondition: ", (fmla_enc(program_ast.postcondition)), "\n\n")

if check_soundness(Q, fmla_enc(program_ast.postcondition)):
    print("Generated postcondition is Consistent and Sound")
else:
    print("Generated postcondition is !Consistent and !Sound")

--- Parsing Structured Program ---
Parsing successful!

--- Precondition ---
And(And(And(And(UGT(msgQSize, 0), ULT(msgQ.fill, msgQSize)),
            Not(isFull(msgQ) == True)),
        msgQ == getRecvrMsgQ(msg.toCore)),
    atomicity == enableAtomicity(msg.toCore))

Generated Postconditions

And(atomicity_0 == disableAtomicity(msg.toCore),
    And(notifyCore(msg.toCore) == True,
        And(msgQ.fill_0 == URem(msgQ.fill_0 + 1, msgQSize),
            And(messageInRB(msgQ, msg) == True,
                And(msgQ.messages_0 == enqueueMsg(msgQ, msg),
                    And(And(And(And(UGT(msgQSize, 0),
                                    ULT(msgQ.fill_0,
                                        msgQSize)),
                                Not(isFull(msgQ) == True)),
                            msgQ == getRecvrMsgQ(msg.toCore)),
                        atomicity_0 ==
                        enableAtomicity(msg.toCore))))))) 



Expected Postcondition:  And(And(msgQ == getRecvrMsgQ(msg.toCore